# Data Loading - All Datasets
## Loading all CSV files from data folder

---
## Setup

In [40]:
import pandas as pd
import numpy as np
import sqlite3
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("✓ Libraries imported")
print(f"Working directory: {os.getcwd()}")

✓ Libraries imported
Working directory: /Users/lorenzogarduno/Documents/datasci:vis/wrangling and transformation


---
## Load All CSV Files

In [41]:
# Define all data sources
data_sources = {
    'billboard': "../data/billboard_24years_lyrics_spotify.csv",
    'musicoset_songs': "../data/musicoset_metadata/songs.csv",
    'musicoset_artists': "../data/musicoset_metadata/artists.csv",
    'musicoset_acoustic': "../data/musicoset_songfeatures/acoustic_features.csv",
    'musicoset_lyrics': "../data/musicoset_songfeatures/lyrics.csv"
}

# Dictionary to store all datasets
datasets = {}
delimiters_used = {}

print("=" * 80)
print("LOADING ALL DATASETS")
print("=" * 80)
print()

# Try different delimiters
delimiters_to_try = [
    (',', 'comma'),
    ('\t', 'tab'),
    (r'\s+', 'whitespace (regex)'),
    ('|', 'pipe')
]

# Load each dataset with delimiter detection
for name, path in data_sources.items():
    if os.path.exists(path):
        loaded = False
        
        # Special case: force acoustic to use tab
        if name == 'musicoset_acoustic':
            try:
                df = pd.read_csv(path, sep='\t')
                datasets[name] = df
                delimiters_used[name] = 'tab'
                print(f"✓ {name:25s}: {df.shape[0]:6d} rows, {df.shape[1]:3d} columns (sep: tab)")
                loaded = True
            except Exception as e:
                datasets[name] = None
                print(f"✗ {name:25s}: Failed with tab delimiter")
                loaded = True
        else:
            # Auto-detect for other files
            for delimiter, delim_name in delimiters_to_try:
                try:
                    # Try loading with this delimiter
                    if delimiter == r'\s+':
                        # Use regex for multiple spaces
                        df = pd.read_csv(path, sep=delimiter, engine='python')
                    else:
                        df = pd.read_csv(path, sep=delimiter)
                    
                    datasets[name] = df
                    delimiters_used[name] = delim_name
                    print(f"✓ {name:25s}: {df.shape[0]:6d} rows, {df.shape[1]:3d} columns (sep: {delim_name})")
                    loaded = True
                    break
                except Exception as e:
                    continue
        
        if not loaded:
            datasets[name] = None
            print(f"✗ {name:25s}: Failed with all delimiters")
    else:
        datasets[name] = None
        print(f"✗ {name:25s}: Not found")

print()
print("=" * 80)
print(f"Total datasets loaded: {sum(1 for v in datasets.values() if v is not None)}")
print("=" * 80)

LOADING ALL DATASETS

✓ billboard                :   3397 rows,  26 columns (sep: comma)
✓ musicoset_songs          :  20405 rows,   7 columns (sep: tab)
✓ musicoset_artists        :  11518 rows,   8 columns (sep: tab)
✓ musicoset_acoustic       :  20405 rows,  14 columns (sep: tab)
✓ musicoset_lyrics         :  20404 rows,   2 columns (sep: tab)

Total datasets loaded: 5


---
## Create Variable Names

In [42]:
# Create convenient variable names
df_billboard = datasets['billboard']
df_songs = datasets['musicoset_songs']
df_artists = datasets['musicoset_artists']
df_acoustic = datasets['musicoset_acoustic']
df_lyrics = datasets['musicoset_lyrics']

print("\nDatasets available as:")
print(f"  - df_billboard  : Billboard 24 years data {'✓' if df_billboard is not None else '✗'}")
print(f"  - df_songs      : MusicoSet songs metadata {'✓' if df_songs is not None else '✗'}")
print(f"  - df_artists    : MusicoSet artists metadata {'✓' if df_artists is not None else '✗'}")
print(f"  - df_acoustic   : MusicoSet acoustic features {'✓' if df_acoustic is not None else '✗'}")
print(f"  - df_lyrics     : MusicoSet lyrics {'✓' if df_lyrics is not None else '✗'}")


Datasets available as:
  - df_billboard  : Billboard 24 years data ✓
  - df_songs      : MusicoSet songs metadata ✓
  - df_artists    : MusicoSet artists metadata ✓
  - df_acoustic   : MusicoSet acoustic features ✓
  - df_lyrics     : MusicoSet lyrics ✓


---
## Load Data into SQLite Database

In [43]:
# Create in-memory SQLite database
conn = sqlite3.connect(':memory:')

print("Loading datasets into SQLite database...\n")

# Load each dataset into SQLite
table_names = {}
for name, df in datasets.items():
    if df is not None:
        table_name = name  # Use dataset name as table name
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        table_names[name] = table_name
        print(f"✓ {name:25s} → table '{table_name}'")

print(f"\n✓ All datasets loaded into SQLite database")
print(f"Total tables: {len(table_names)}")

Loading datasets into SQLite database...

✓ billboard                 → table 'billboard'
✓ musicoset_songs           → table 'musicoset_songs'
✓ musicoset_artists         → table 'musicoset_artists'
✓ musicoset_acoustic        → table 'musicoset_acoustic'
✓ musicoset_lyrics          → table 'musicoset_lyrics'

✓ All datasets loaded into SQLite database
Total tables: 5


---
## Dataset Schema Inspection
### Examine the structure of Billboard and MusicoSet Songs tables

In [44]:
# Examine all relevant table structures
print("=" * 80)
print("BILLBOARD TABLE SCHEMA")
print("=" * 80)
billboard_info = pd.read_sql_query("PRAGMA table_info(billboard)", conn)
print(billboard_info.to_string(index=False))
print(f"\nColumns: {list(df_billboard.columns)}")

print("\n" + "=" * 80)
print("MUSICOSET SONGS TABLE SCHEMA")
print("=" * 80)
songs_info = pd.read_sql_query("PRAGMA table_info(musicoset_songs)", conn)
print(songs_info.to_string(index=False))
print(f"\nColumns: {list(df_songs.columns)}")

print("\n" + "=" * 80)
print("MUSICOSET ARTISTS TABLE SCHEMA")
print("=" * 80)
artists_info = pd.read_sql_query("PRAGMA table_info(musicoset_artists)", conn)
print(artists_info.to_string(index=False))
print(f"\nColumns: {list(df_artists.columns)}")

# Show sample data to understand the data structure
print("\n" + "=" * 80)
print("BILLBOARD SAMPLE DATA (first 3 rows, key columns)")
print("=" * 80)
billboard_sample = pd.read_sql_query("SELECT ranking, song, band_singer, year FROM billboard LIMIT 3", conn)
print(billboard_sample.to_string(index=False))

print("\n" + "=" * 80)
print("MUSICOSET SONGS SAMPLE DATA (first 3 rows)")
print("=" * 80)
songs_sample = pd.read_sql_query("SELECT * FROM musicoset_songs LIMIT 3", conn)
print(songs_sample.to_string(index=False))

print("\n" + "=" * 80)
print("MUSICOSET ARTISTS SAMPLE DATA (first 3 rows)")
print("=" * 80)
artists_sample = pd.read_sql_query("SELECT * FROM musicoset_artists LIMIT 3", conn)
print(artists_sample.to_string(index=False))

BILLBOARD TABLE SCHEMA
 cid             name    type  notnull dflt_value  pk
   0          ranking INTEGER        0       None   0
   1             song    TEXT        0       None   0
   2      band_singer    TEXT        0       None   0
   3          songurl    TEXT        0       None   0
   4        titletext    TEXT        0       None   0
   5              url    TEXT        0       None   0
   6             year INTEGER        0       None   0
   7           lyrics    TEXT        0       None   0
   8              uri    TEXT        0       None   0
   9     danceability    REAL        0       None   0
  10           energy    REAL        0       None   0
  11              key    REAL        0       None   0
  12         loudness    REAL        0       None   0
  13             mode    REAL        0       None   0
  14      speechiness    REAL        0       None   0
  15     acousticness    REAL        0       None   0
  16 instrumentalness    REAL        0       None   0
  17 

---
## Create Combined Dataset: Billboard + Song Genres
### Join Billboard data with MusicoSet Songs on song name and year to add genre information

In [45]:
# SQL JOIN query to combine Billboard with MusicoSet Songs and Artists
# Using LEFT JOIN to preserve all Billboard records

sql_query = """
SELECT 
    b.ranking AS billboard_position,
    b.song AS song_name,
    b.band_singer AS artist,
    b.year AS year,
    a.main_genre AS genre,
    a.genres AS genres,
    s.popularity AS song_popularity,
    s.song_type AS song_type
FROM billboard b
LEFT JOIN musicoset_artists a
    ON LOWER(TRIM(b.band_singer)) = LOWER(TRIM(a.name))
LEFT JOIN musicoset_songs s
    ON LOWER(TRIM(b.song)) = LOWER(TRIM(s.song_name))
ORDER BY b.year, b.ranking
"""

print("=" * 80)
print("EXECUTING SQL JOIN QUERY")
print("=" * 80)
print("\nQuery:")
print(sql_query)
print("\nUsing LEFT JOIN to preserve all Billboard records")
print("Genre info will be added where matches are found")

# Execute the query and create the new dataset
df_billboard_genres = pd.read_sql_query(sql_query, conn)

print("\n" + "=" * 80)
print("JOINED DATASET CREATED")
print("=" * 80)
print(f"Total rows: {len(df_billboard_genres)}")
print(f"Total columns: {len(df_billboard_genres.columns)}")
print(f"\nColumns: {list(df_billboard_genres.columns)}")

# Show first 10 rows of the new dataset
print("\n" + "=" * 80)
print("SAMPLE DATA (first 10 rows)")
print("=" * 80)
print(df_billboard_genres.head(10).to_string(index=False))

EXECUTING SQL JOIN QUERY

Query:

SELECT 
    b.ranking AS billboard_position,
    b.song AS song_name,
    b.band_singer AS artist,
    b.year AS year,
    a.main_genre AS genre,
    a.genres AS genres,
    s.popularity AS song_popularity,
    s.song_type AS song_type
FROM billboard b
LEFT JOIN musicoset_artists a
    ON LOWER(TRIM(b.band_singer)) = LOWER(TRIM(a.name))
LEFT JOIN musicoset_songs s
    ON LOWER(TRIM(b.song)) = LOWER(TRIM(s.song_name))
ORDER BY b.year, b.ranking


Using LEFT JOIN to preserve all Billboard records
Genre info will be added where matches are found



JOINED DATASET CREATED
Total rows: 4686
Total columns: 8

Columns: ['billboard_position', 'song_name', 'artist', 'year', 'genre', 'genres', 'song_popularity', 'song_type']

SAMPLE DATA (first 10 rows)
 billboard_position    song_name          artist  year                genre                                                                              genres  song_popularity     song_type
                  1      Breathe      Faith Hill  2000 contemporary country     ['contemporary country', 'country', 'country dawn', 'country road', 'pop rock']             51.0          Solo
                  1      Breathe      Faith Hill  2000 contemporary country     ['contemporary country', 'country', 'country dawn', 'country road', 'pop rock']             52.0          Solo
                  1      Breathe      Faith Hill  2000 contemporary country     ['contemporary country', 'country', 'country dawn', 'country road', 'pop rock']             59.0 Collaboration
                  1      Breathe  

In [46]:
# Analysis of the JOIN results
print("=" * 80)
print("JOIN STATISTICS")
print("=" * 80)

# Count original records
total_billboard = pd.read_sql_query("SELECT COUNT(*) as count FROM billboard", conn).iloc[0, 0]
total_artists = pd.read_sql_query("SELECT COUNT(*) as count FROM musicoset_artists", conn).iloc[0, 0]
total_songs = pd.read_sql_query("SELECT COUNT(*) as count FROM musicoset_songs", conn).iloc[0, 0]
total_matched = len(df_billboard_genres)

print(f"\nOriginal Billboard records: {total_billboard}")
print(f"Original MusicoSet Artists records: {total_artists}")
print(f"Original MusicoSet Songs records: {total_songs}")
print(f"Matched records (joined): {total_matched}")
print(f"Match rate: {(total_matched / total_billboard * 100):.2f}% of Billboard songs")

# Check how many have genre information
with_genre = df_billboard_genres[df_billboard_genres['genre'].notna()]
print(f"Records with genre information: {len(with_genre)} ({len(with_genre) / len(df_billboard_genres) * 100:.2f}%)")

# Show distribution by year
print("\n" + "=" * 80)
print("MATCHED SONGS BY YEAR")
print("=" * 80)
year_distribution = df_billboard_genres.groupby('year').size().reset_index(name='count')
print(year_distribution.to_string(index=False))

# Show genre distribution
print("\n" + "=" * 80)
print("GENRE DISTRIBUTION (top 15)")
print("=" * 80)
genre_distribution = df_billboard_genres[df_billboard_genres['genre'].notna()].groupby('genre').size().reset_index(name='count').sort_values('count', ascending=False).head(15)
print(genre_distribution.to_string(index=False))

JOIN STATISTICS

Original Billboard records: 3397
Original MusicoSet Artists records: 11518
Original MusicoSet Songs records: 20405
Matched records (joined): 4686
Match rate: 137.95% of Billboard songs
Records with genre information: 4224 (90.14%)

MATCHED SONGS BY YEAR
 year  count
 2000    165
 2001    216
 2002    200
 2003    206
 2004    211
 2005    192
 2006    193
 2007    217
 2008    203
 2009    192
 2010    224
 2011    173
 2012    192
 2013    208
 2014    205
 2015    209
 2016    198
 2017    214
 2018    223
 2019    188
 2020    158
 2021    176
 2022    171
 2023    152

GENRE DISTRIBUTION (top 15)
                    genre  count
                dance pop   1349
     contemporary country    321
              atl hip hop    319
                      pop    223
                        -    175
                  hip hop    129
        alternative metal    129
         canadian hip hop     97
             canadian pop     95
              chicago rap     81
           a

---
## Dataset Creation

Creating datasets with artist-level granularity and lyrics-based matching

In [47]:
# Helper functions for artist parsing and lyrics matching

def parse_artists(artist_string):
    """Extract individual artists from collaboration strings"""
    if pd.isna(artist_string):
        return []
    
    text = str(artist_string)
    
    # Split patterns (in order of specificity)
    patterns = [
        (r'\s+featuring\s+', 'featuring'),
        (r'\s+feat\.?\s+', 'feat'),
        (r'\s+ft\.?\s+', 'ft'),
        (r'\s+[Xx]\s+', 'x'),
        (r'\s*&\s*', '&'),
        (r'\s*,\s*', ',')
    ]
    
    # Try each pattern
    for pattern, _ in patterns:
        if re.search(pattern, text, re.IGNORECASE):
            artists = re.split(pattern, text, flags=re.IGNORECASE)
            # Clean up
            artists = [a.strip() for a in artists if a.strip()]
            # Remove common suffixes like "and others"
            artists = [a for a in artists if not re.match(r'^(and|with|plus)\s', a, re.IGNORECASE)]
            return artists[:5]  # Max 5 artists to avoid noise
    
    # No pattern found - single artist
    return [text.strip()]


def calc_lyrics_similarity(lyrics1, lyrics2):
    """Calculate Jaccard similarity between two lyrics (word-level)"""
    if pd.isna(lyrics1) or pd.isna(lyrics2):
        return 0.0
    
    # Tokenize into words
    words1 = set(re.findall(r'\b[a-z]+\b', str(lyrics1).lower()))
    words2 = set(re.findall(r'\b[a-z]+\b', str(lyrics2).lower()))
    
    if not words1 or not words2:
        return 0.0
    
    # Jaccard similarity
    intersection = len(words1 & words2)
    union = len(words1 | words2)
    
    return intersection / union if union > 0 else 0.0


print("Helper functions defined")
print("\nTesting artist parsing:")
test_cases = [
    "Santana featuring Rob Thomas",
    "Migos, Nicki Minaj & Cardi B",
    "Drake",
    "Post Malone feat. Swae Lee"
]
for case in test_cases:
    print(f"  '{case}' → {parse_artists(case)}")

Helper functions defined

Testing artist parsing:
  'Santana featuring Rob Thomas' → ['Santana', 'Rob Thomas']
  'Migos, Nicki Minaj & Cardi B' → ['Migos, Nicki Minaj', 'Cardi B']
  'Drake' → ['Drake']
  'Post Malone feat. Swae Lee' → ['Post Malone', 'Swae Lee']


In [48]:
# Dataset 1: billboard_master (with deduplication)

sql = """
WITH ranked AS (
    SELECT 
        b.ranking, b.song, b.band_singer, b.year, b.lyrics,
        a.main_genre, a.genres,
        ROW_NUMBER() OVER (PARTITION BY b.song, b.band_singer, b.year ORDER BY a.main_genre) as rn
    FROM billboard b
    LEFT JOIN musicoset_artists a ON LOWER(TRIM(b.band_singer)) = LOWER(TRIM(a.name))
)
SELECT ranking, song, band_singer, year, lyrics, main_genre, genres
FROM ranked
WHERE rn = 1 OR rn IS NULL
"""

df_master = pd.read_sql_query(sql, conn)

# Lexical metrics
def calc_metrics(lyrics):
    if pd.isna(lyrics):
        return pd.Series([0, 0, 0.0])
    words = re.findall(r'\b[a-z]+\b', str(lyrics).lower())
    if not words:
        return pd.Series([0, 0, 0.0])
    return pd.Series([len(words), len(set(words)), len(set(words))/len(words)])

df_master[['word_count', 'unique_words', 'ttr']] = df_master['lyrics'].apply(calc_metrics)

# Count total artists per song
df_master['artist_list'] = df_master['band_singer'].apply(parse_artists)
df_master['num_artists'] = df_master['artist_list'].apply(len)
df_master['is_collab'] = df_master['num_artists'] > 1

print(f"billboard_master: {len(df_master)} rows")
print(f"  Collaborations: {df_master['is_collab'].sum()} ({df_master['is_collab'].sum()/len(df_master)*100:.1f}%)")
print(f"  Max artists on one song: {df_master['num_artists'].max()}")
df_master.head()

billboard_master: 3397 rows
  Collaborations: 23 (0.7%)
  Max artists on one song: 2


,ranking,song,band_singer,year,lyrics,main_genre,genres,word_count,unique_words,ttr,artist_list,num_artists,is_collab
0,81,#Beautiful,Mariah Carey,2013,"Ah, ah, you’re beautiful (Yeah)\nAh, ah, you’r...",dance pop,"['dance pop', 'pop', 'r&b', 'urban contemporary']",305.0,89.0,0.291803,[Mariah Carey],1,False
1,81,#Beautiful,Miguel,2013,"Ah, ah, you’re beautiful (Yeah)\nAh, ah, you’r...",dance pop,"['dance pop', 'pop', 'pop rap', 'r&b', 'rap', ...",305.0,89.0,0.291803,[Miguel],1,False
2,95,#thatPower,Justin Bieber,2013,"\nAnd, oh\nI'm alive, I'm alive, I'm alive\nAn...",canadian pop,"['canadian pop', 'dance pop', 'pop', 'post-tee...",450.0,125.0,0.277778,[Justin Bieber],1,False
3,95,#thatPower,will.i.am,2013,"\nAnd, oh\nI'm alive, I'm alive, I'm alive\nAn...",dance pop,"['dance pop', 'pop', 'pop rap', 'post-teen pop']",450.0,125.0,0.277778,[will.i.am],1,False
4,37,'03 Bonnie & Clyde,Beyoncé,2003,"Uh-huh, uh, uh\nYou ready, B?\nLet's go get 'e...",dance pop,"['dance pop', 'pop', 'r&b']",672.0,213.0,0.316964,[Beyoncé],1,False


In [49]:
# Lyrics-based matching using TF-IDF (fast & accurate)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

print("\nAttempting lyrics-based matching (TF-IDF + Cosine Similarity)...")

# Get unmatched songs and potential matches
unmatched = df_song_artists[df_song_artists['artist_genre'].isna() & df_song_artists['lyrics'].notna()].copy()
matched_pool = df_master[df_master['main_genre'].notna()][['lyrics', 'main_genre', 'song', 'band_singer']].copy()

if len(unmatched) > 0:
    print(f"  Unmatched songs with lyrics: {len(unmatched)}")
    print(f"  Candidate pool: {len(matched_pool)} songs")
    print(f"  Processing...")

    # TF-IDF vectorization
    vectorizer = TfidfVectorizer(
        lowercase=True,
        token_pattern=r'\b[a-z]+\b',  # Only alphabetic words
        min_df=1,  # Word must appear in at least 1 song
        max_df=0.95,  # Ignore words in >95% of songs (too common)
        max_features=5000  # Limit vocabulary size
    )

    # Fit vectorizer on all lyrics
    all_lyrics = unmatched['lyrics'].fillna('').tolist() + matched_pool['lyrics'].fillna('').tolist()
    vectorizer.fit(all_lyrics)

    # Transform to vectors
    unmatched_vectors = vectorizer.transform(unmatched['lyrics'].fillna(''))
    matched_vectors = vectorizer.transform(matched_pool['lyrics'].fillna(''))

    # Compute similarity matrix: unmatched × matched
    # This is the magic - one matrix operation instead of nested loops
    similarity_matrix = cosine_similarity(unmatched_vectors, matched_vectors)

    # Find best match for each unmatched song
    matches_found = 0
    for i, (idx, row) in enumerate(unmatched.iterrows()):
        best_match_idx = np.argmax(similarity_matrix[i])
        best_score = similarity_matrix[i, best_match_idx]

        # Apply 60% threshold
        if best_score >= 0.60:
            matched_info = matched_pool.iloc[best_match_idx]
            df_song_artists.loc[idx, 'artist_genre'] = matched_info['main_genre']
            df_song_artists.loc[idx, 'match_method'] = f'lyrics_{best_score:.2f}'
            matches_found += 1

    print(f"  ✓ Matched {matches_found} songs above 60% threshold")
    
    # Show some examples
    if matches_found > 0:
        examples = df_song_artists[df_song_artists['match_method'].str.startswith('lyrics', na=False)].head(3)
        print(f"\n  Example matches:")
        for _, ex in examples.iterrows():
            print(f"    • {ex['song_name']} by {ex['artist_name']} → {ex['artist_genre']} ({ex['match_method']})")
else:
    print("  All songs already matched by artist name!")

# Add match_method for artist-matched songs
df_song_artists['match_method'] = df_song_artists['match_method'].fillna('artist_name')

# Final stats
matched_count = df_song_artists['artist_genre'].notna().sum()
total_count = len(df_song_artists)
print(f"\n  Final coverage: {matched_count} / {total_count} ({matched_count/total_count*100:.1f}%)")
print(f"  • Matched by artist name: {(df_song_artists['match_method'] == 'artist_name').sum()}")
print(f"  • Matched by lyrics: {(df_song_artists['match_method'].str.startswith('lyrics', na=False)).sum()}")

df_song_artists.head(10)


Attempting lyrics-based matching (TF-IDF + Cosine Similarity)...
  Unmatched songs with lyrics: 403
  Candidate pool: 3020 songs
  Processing...
  ✓ Matched 198 songs above 60% threshold

  Example matches:
    • '03 Bonnie & Clyde by Jay-Z → dance pop (lyrics_1.00)
    • 34+35 by Megan Thee Stallion → dance pop (lyrics_0.82)
    • ABCDEFU by Gayle → dance pop (lyrics_0.73)

  Final coverage: 3215 / 3420 (94.0%)
  • Matched by artist name: 3222
  • Matched by lyrics: 198


,song_id,song_name,year,ranking,artist_name,artist_position,artist_role,lyrics,word_count,artist_genre,artist_genres,match_method
0,0,#Beautiful,2013,81,Mariah Carey,1,primary,"Ah, ah, you’re beautiful (Yeah)\nAh, ah, you’r...",305.0,dance pop,"['dance pop', 'pop', 'r&b', 'urban contemporary']",artist_name
1,1,#Beautiful,2013,81,Miguel,1,primary,"Ah, ah, you’re beautiful (Yeah)\nAh, ah, you’r...",305.0,dance pop,"['dance pop', 'pop', 'pop rap', 'r&b', 'rap', ...",artist_name
2,2,#thatPower,2013,95,Justin Bieber,1,primary,"\nAnd, oh\nI'm alive, I'm alive, I'm alive\nAn...",450.0,canadian pop,"['canadian pop', 'dance pop', 'pop', 'post-tee...",artist_name
3,3,#thatPower,2013,95,will.i.am,1,primary,"\nAnd, oh\nI'm alive, I'm alive, I'm alive\nAn...",450.0,dance pop,"['dance pop', 'pop', 'pop rap', 'post-teen pop']",artist_name
4,4,'03 Bonnie & Clyde,2003,37,Beyoncé,1,primary,"Uh-huh, uh, uh\nYou ready, B?\nLet's go get 'e...",672.0,dance pop,"['dance pop', 'pop', 'r&b']",artist_name
5,5,'03 Bonnie & Clyde,2003,37,Jay-Z,1,primary,"Uh-huh, uh, uh\nYou ready, B?\nLet's go get 'e...",672.0,dance pop,None,lyrics_1.00
6,6,'Til You Can't,2022,34,Cody Johnson,1,primary,You can tell your old man you'll do some large...,390.0,contemporary country,"['contemporary country', 'country road', 'mode...",artist_name
7,7,(When You Gonna) Give It Up to Me,2006,37,Keyshia Cole,1,primary,"Get out my head and into the bed, girl\nCau ya...",954.0,dance pop,"['dance pop', 'deep pop r&b', 'hip pop', 'neo ...",artist_name
8,8,(When You Gonna) Give It Up to Me,2006,37,Sean Paul,1,primary,"Get out my head and into the bed, girl\nCau ya...",954.0,-,[],artist_name
9,9,0 to 100 / The Catch Up,2014,97,Drake,1,primary,\nFuck bein' on some chill shit\nWe go 0 to 10...,1013.0,canadian hip hop,"['canadian hip hop', 'canadian pop', 'hip hop'...",artist_name


In [50]:
# Dataset 3: temporal_trends (year × genre aggregations)

df_trends = df_master[df_master['main_genre'].notna()].groupby(['year', 'main_genre']).agg({
    'song': 'count',
    'word_count': 'mean',
    'ttr': 'mean',
    'is_collab': 'sum',
    'num_artists': 'mean'
}).reset_index()

df_trends.columns = ['year', 'genre', 'song_count', 'avg_words', 'avg_ttr', 'collab_count', 'avg_artists']
df_trends['genre_pct'] = df_trends.groupby('year')['song_count'].transform(lambda x: (x / x.sum() * 100).round(1))
df_trends['collab_rate'] = (df_trends['collab_count'] / df_trends['song_count'] * 100).round(1)

print(f"temporal_trends: {len(df_trends)} rows")
df_trends.head(10)

temporal_trends: 754 rows


,year,genre,song_count,avg_words,avg_ttr,collab_count,avg_artists,genre_pct,collab_rate
0,2000,-,5,538.000000,0.222878,1,1.2,5.1,20.0
1,2000,acoustic pop,1,388.000000,0.298969,0,1.0,1.0,0.0
2,2000,alternative hip hop,1,881.000000,0.338252,0,1.0,1.0,0.0
3,2000,alternative metal,6,352.666667,0.311609,0,1.0,6.1,0.0
4,2000,alternative rock,3,326.000000,0.292127,0,1.0,3.1,0.0
5,2000,atl hip hop,3,709.000000,0.222104,0,1.0,3.1,0.0
6,2000,australian pop,3,420.666667,0.221820,0,1.0,3.1,0.0
7,2000,blues-rock,2,400.500000,0.321397,0,1.0,2.0,0.0
8,2000,boy band,5,463.200000,0.263260,0,1.0,5.1,0.0
9,2000,bubblegum dance,1,613.000000,0.123980,0,1.0,1.0,0.0


In [51]:
# Dataset 4: artist_profiles (per-artist stats)

df_artist_profiles = df_master.groupby('band_singer').agg({
    'year': ['min', 'max', 'count'],
    'ranking': ['mean', 'min'],
    'main_genre': lambda x: x.mode()[0] if len(x.mode()) > 0 else None,
    'word_count': 'mean',
    'ttr': 'mean',
    'is_collab': 'sum',
    'num_artists': 'mean'
}).reset_index()

df_artist_profiles.columns = ['artist', 'first_year', 'last_year', 'appearances', 
                               'avg_rank', 'best_rank', 'main_genre', 'avg_words', 'avg_ttr', 'collabs', 'avg_collab_size']
df_artist_profiles['years_active'] = df_artist_profiles['last_year'] - df_artist_profiles['first_year'] + 1

print(f"artist_profiles: {len(df_artist_profiles)} rows")
df_artist_profiles.sort_values('appearances', ascending=False).head(10)

artist_profiles: 1039 rows


,artist,first_year,last_year,appearances,avg_rank,best_rank,main_genre,avg_words,avg_ttr,collabs,avg_collab_size,years_active
264,Drake,2009,2023,70,48.514286,1,canadian hip hop,630.371429,0.329500,0,1.0,15
799,Rihanna,2005,2023,46,40.173913,2,dance pop,530.326087,0.260930,0,1.0,19
179,Chris Brown,2005,2023,36,50.055556,6,dance pop,692.027778,0.254454,0,1.0,19
569,Lil Wayne,2005,2020,35,50.000000,4,hip hop,709.514286,0.348659,0,1.0,16
905,Taylor Swift,2007,2023,34,43.588235,4,dance pop,439.294118,0.302401,0,1.0,17
700,Nicki Minaj,2010,2023,34,55.382353,8,dance pop,641.647059,0.303714,0,1.0,14
473,Justin Bieber,2009,2022,32,38.656250,1,canadian pop,471.687500,0.294075,0,1.0,14
589,Ludacris,2001,2011,29,51.724138,1,atl hip hop,685.655172,0.319304,0,1.0,11
90,Beyoncé,2003,2023,27,45.296296,1,dance pop,603.296296,0.286988,0,1.0,21
976,Usher,2001,2016,26,42.076923,1,atl hip hop,551.269231,0.276908,0,1.0,16


In [52]:
# Dataset 5: genre_network (genre co-occurrence)

pairs = []
for year in df_master['year'].unique():
    genres = df_master[(df_master['year'] == year) & (df_master['main_genre'].notna())]['main_genre'].unique()
    for i, g1 in enumerate(genres):
        for g2 in genres[i+1:]:
            pairs.append({'year': year, 'genre1': g1, 'genre2': g2})

df_network = pd.DataFrame(pairs).groupby(['genre1', 'genre2']).agg({'year': ['count', 'min', 'max']}).reset_index()
df_network.columns = ['genre1', 'genre2', 'years_together', 'first_year', 'last_year']

print(f"genre_network: {len(df_network)} rows")
df_network.sort_values('years_together', ascending=False).head(15)

genre_network: 6316 rows


,genre1,genre2,years_together,first_year,last_year
2866,dance pop,atl hip hop,23,2000,2023
2853,dance pop,-,22,2000,2023
2988,dance pop,pop,19,2002,2023
2893,dance pop,canadian pop,18,2001,2022
2536,contemporary country,-,17,2000,2023
2908,dance pop,contemporary country,17,2002,2021
2919,dance pop,detroit hip hop,17,2000,2020
2901,dance pop,chicago rap,17,2000,2021
2871,dance pop,australian pop,17,2000,2019
2951,dance pop,hip hop,16,2000,2023


In [53]:
# Export all datasets

# Clean up columns before export
df_master_export = df_master.drop(columns=['artist_list'], errors='ignore')
df_song_artists_export = df_song_artists.drop(columns=['lyrics'], errors='ignore')  # Don't export lyrics in song_artists

df_master_export.to_csv('../data/billboard_master.csv', index=False)
df_song_artists_export.to_csv('../data/song_artists.csv', index=False)
df_trends.to_csv('../data/temporal_trends.csv', index=False)
df_artist_profiles.to_csv('../data/artist_profiles.csv', index=False)
df_network.to_csv('../data/genre_network.csv', index=False)

print("Exported:")
print(f"  billboard_master.csv    - {len(df_master_export)} rows (song-level)")
print(f"  song_artists.csv        - {len(df_song_artists_export)} rows (artist-level) ⭐ NEW")
print(f"  temporal_trends.csv     - {len(df_trends)} rows (year × genre)")
print(f"  artist_profiles.csv     - {len(df_artist_profiles)} rows (per artist)")
print(f"  genre_network.csv       - {len(df_network)} rows (genre pairs)")
print(f"\n✓ All datasets created with artist parsing + lyrics matching (60% threshold)")

Exported:
  billboard_master.csv    - 3397 rows (song-level)
  song_artists.csv        - 3420 rows (artist-level) ⭐ NEW
  temporal_trends.csv     - 754 rows (year × genre)
  artist_profiles.csv     - 1039 rows (per artist)
  genre_network.csv       - 6316 rows (genre pairs)

✓ All datasets created with artist parsing + lyrics matching (60% threshold)


In [54]:
df_master.to_csv('../data/billboard_master.csv', index=False)
df_trends.to_csv('../data/temporal_trends.csv', index=False)
df_artists.to_csv('../data/artist_profiles.csv', index=False)
df_network.to_csv('../data/genre_network.csv', index=False)

print("Exported:")
print(f"  billboard_master.csv    - {len(df_master)} rows")
print(f"  temporal_trends.csv     - {len(df_trends)} rows")
print(f"  artist_profiles.csv     - {len(df_artists)} rows")
print(f"  genre_network.csv       - {len(df_network)} rows")

Exported:
  billboard_master.csv    - 3397 rows
  temporal_trends.csv     - 754 rows
  artist_profiles.csv     - 11518 rows
  genre_network.csv       - 6316 rows


In [55]:
# Dataset 4: genre_network
# Genre co-occurrence by year

pairs = []
for year in df_master['year'].unique():
    genres = df_master[(df_master['year'] == year) & (df_master['main_genre'].notna())]['main_genre'].unique()
    for i, g1 in enumerate(genres):
        for g2 in genres[i+1:]:
            pairs.append({'year': year, 'genre1': g1, 'genre2': g2})

df_network = pd.DataFrame(pairs).groupby(['genre1', 'genre2']).agg({
    'year': ['count', 'min', 'max']
}).reset_index()

df_network.columns = ['genre1', 'genre2', 'years_together', 'first_year', 'last_year']

print(f"Dataset 4: {len(df_network)} rows")
df_network.sort_values('years_together', ascending=False).head(15)

Dataset 4: 6316 rows


,genre1,genre2,years_together,first_year,last_year
2866,dance pop,atl hip hop,23,2000,2023
2853,dance pop,-,22,2000,2023
2988,dance pop,pop,19,2002,2023
2893,dance pop,canadian pop,18,2001,2022
2536,contemporary country,-,17,2000,2023
2908,dance pop,contemporary country,17,2002,2021
2919,dance pop,detroit hip hop,17,2000,2020
2901,dance pop,chicago rap,17,2000,2021
2871,dance pop,australian pop,17,2000,2019
2951,dance pop,hip hop,16,2000,2023


In [56]:
# Dataset 2: temporal_trends
# Year and genre aggregations

df_trends = df_master[df_master['main_genre'].notna()].groupby(['year', 'main_genre']).agg({
    'song': 'count',
    'word_count': 'mean',
    'ttr': 'mean',
    'is_collab': 'sum'
}).reset_index()

df_trends.columns = ['year', 'genre', 'song_count', 'avg_words', 'avg_ttr', 'collab_count']

# Genre percentage per year
df_trends['genre_pct'] = df_trends.groupby('year')['song_count'].transform(
    lambda x: (x / x.sum() * 100).round(1)
)

print(f"Dataset 2: {len(df_trends)} rows")
df_trends.head(10)

Dataset 2: 754 rows


,year,genre,song_count,avg_words,avg_ttr,collab_count,genre_pct
0,2000,-,5,538.000000,0.222878,1,5.1
1,2000,acoustic pop,1,388.000000,0.298969,0,1.0
2,2000,alternative hip hop,1,881.000000,0.338252,0,1.0
3,2000,alternative metal,6,352.666667,0.311609,0,6.1
4,2000,alternative rock,3,326.000000,0.292127,0,3.1
5,2000,atl hip hop,3,709.000000,0.222104,0,3.1
6,2000,australian pop,3,420.666667,0.221820,0,3.1
7,2000,blues-rock,2,400.500000,0.321397,0,2.0
8,2000,boy band,5,463.200000,0.263260,0,5.1
9,2000,bubblegum dance,1,613.000000,0.123980,0,1.0
